In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import glob

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import itertools
import shutil
import matplotlib.pyplot as plt
from numpy.random import seed
np.random.seed(101)
tf.random.set_seed(101)

print(tf.__version__)

from tensorflow.keras.layers import Dropout, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


2.12.0


In [ ]:
train_path = 'gdrive/My Drive/CV_Project/train'
valid_path = 'gdrive/My Drive/CV_Project/test'

num_train_samples = 800
num_val_samples = 160
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

print(int(train_steps))
print(int(val_steps))


80
16


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
   
    preprocessing_function= preprocess_input)


train_batches = datagen.flow_from_directory(train_path,
                                           target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical',
                                            subset='training')

valid_batches = datagen.flow_from_directory(train_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical',
                                            subset='training')

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

Found 833 images belonging to 4 classes.
Found 833 images belonging to 4 classes.
Found 160 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Create an ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
     validation_split=0.2,
)

# Load and preprocess training images
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load and preprocess validation images
validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)




Found 667 images belonging to 4 classes.
Found 166 images belonging to 4 classes.


In [ ]:
import os
from PIL import Image

# Replace 'image_directory' with the name of the extracted folder in Colab
directory = 'gdrive/My Drive/CV_Project/train/chives_images'

# List of supported image file extensions
supported_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"]

for file_name in os.listdir(directory):
    
    # Check if the file has a supported image extension
    if any(file_name.lower().endswith(ext) for ext in supported_extensions):
        file_path = os.path.join(directory, file_name)

        try:
            # Attempt to open the image with PIL
            with Image.open(file_path) as img:
                a=2
                # print(f"{file_name} is a valid image.")
        except IOError:
            print(f"{file_name} is not a valid image or has issues.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

new_model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Replace 4 with the number of classes in your dataset
])

# Compile the model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=12,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

test_generator = datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)



Epoch 1/12
21/21 [==============================] - 135s 7s/step - loss: 1.5532 - accuracy: 0.2624 - val_loss: 1.3732 - val_accuracy: 0.2952
Epoch 2/12
21/21 [==============================] - 44s 2s/step - loss: 1.3577 - accuracy: 0.3313 - val_loss: 1.3287 - val_accuracy: 0.3494
Epoch 3/12
21/21 [==============================] - 44s 2s/step - loss: 1.2996 - accuracy: 0.4123 - val_loss: 1.2667 - val_accuracy: 0.4157
Epoch 4/12
21/21 [==============================] - 44s 2s/step - loss: 1.1883 - accuracy: 0.4813 - val_loss: 1.2212 - val_accuracy: 0.4157
Epoch 5/12
21/21 [==============================] - 49s 2s/step - loss: 1.1460 - accuracy: 0.4903 - val_loss: 1.3257 - val_accuracy: 0.3614
Epoch 6/12
21/21 [==============================] - 44s 2s/step - loss: 1.0478 - accuracy: 0.5802 - val_loss: 1.3134 - val_accuracy: 0.3916
Epoch 7/12
21/21 [==============================] - 44s 2s/step - loss: 0.8265 - accuracy: 0.6837 - val_loss: 1.4189 - val_accuracy: 0.4217
Epoch 8/12
21/21 [=

In [ ]:
new_model.save('herb_classification_model.h5')

metrics = new_model.evaluate_generator(test_generator)
print('Test accuracy:', metrics[1])

<ipython-input-10-406a334e17e4>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  metrics = new_model.evaluate_generator(test_generator)


Test accuracy: 0.4000000059604645


In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  temp=0
  x=0
  c=0
  for i in classes[0]:
    if(i>temp):
     
      temp=i
      x=c
    c+=1
  class_names={}
  class_names={0:'Chives',1:'Oregano',2:'Parsley',3:'Thyme'}
  print(classes)
  print(classes[0])
  print(class_names[x])